In [43]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import faiss

Загрузим модель от DeepPavlov

In [4]:
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

d:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\79169\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
data_train = pd.read_csv(r"C:\Users\79169\Desktop\домашка\DLL\dataset_train.tsv", sep='\t', header=None)
data_train

,0,1
0,мне нужна справка,statement_general
1,оформить справку,statement_general
2,взять справку,statement_general
3,справку как получить,statement_general
4,справку ммф где получаться,statement_general
...,...,...
13225,тупой,smalltalk_abuse
13226,робот бестолковый,smalltalk_abuse
13227,несообразительный,smalltalk_abuse
13228,ты бестолковый,smalltalk_abuse


In [10]:
data_test = pd.read_csv(r"C:\Users\79169\Desktop\домашка\DLL\dataset_test.tsv", sep='\t',header=None)
data_test

,0,1
0,как получить справку,statement_general
1,мне нужна справка,statement_general
2,справка студента эф петь,conform
3,справка студента фф оформлять,conform
4,как мне заказать справка об обучении,conform
...,...,...
878,как получить оплата дороги,travelfinaid_get
879,как получить финансовая поддержка поездки,travelfinaid_get
880,ты бесполезный,smalltalk_abuse
881,бот совершено глупый,smalltalk_abuse


In [48]:
texts = data_train[0]
labels = data_train[1]
test_text = data_test[0]
test_labels = data_test[1]

In [35]:
# Функция для получения эмбеддингов
def get_embeddings(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        # Получаем эмбеддинг первого токена (CLS токен)
        embeddings.append(outputs.last_hidden_state[:,0,:].squeeze().numpy())
    return embeddings

In [36]:
# Получение эмбеддингов
embeddings = get_embeddings(texts)

In [45]:
#перевод их в numpy
embeddings_np = np.array(embeddings).astype('float32') 

In [64]:
from collections import Counter

Для построения векторного поискового индекса и классификации на основе ближайших соседей используем Faiss

In [63]:
# Создание индекса FAISS
d = embeddings_np.shape[1]  # размерность эмбеддингов
# Нормализация эмбеддингов для косинусного сходства
faiss.normalize_L2(embeddings_np)
# Создание индекса для косинусного сходства (используем индекс скалярного произведения)
index = faiss.IndexFlatIP(d)
# Добавление эмбеддингов в индекс
index.add(embeddings_np)

In [65]:
def classify(query_embedding, k=5):
    """
    Классифицирует запрос на основе k наиболее близких соседей.
    """
    # Нормализация запроса
    faiss.normalize_L2(query_embedding.reshape(1, -1))

    # Поиск k наиболее близких соседей
    D, I = index.search(query_embedding.reshape(1, -1), k)

    # Определение наиболее часто встречающегося класса среди соседей
    nearest_labels = labels[I[0]]
    most_common = Counter(nearest_labels).most_common(1)
    return most_common[0][0]  # Возвращает класс наиболее близкого соседа


query_embedding = np.random.rand(d).astype('float32')  # попробуем предсказать на основе вектора запроса
predicted_class = classify(query_embedding, k=5)


In [68]:
print(predicted_class)

student_union_print


Протестируем модель на тестовых данных

In [67]:
embeddings_t = get_embeddings(test_text)
embeddings_np = np.array(embeddings_t).astype('float32') 

In [69]:
predicted_labels = [classify(embedding, k=5) for embedding in embeddings_np]

In [72]:
query_embedding = np.random.rand(d).astype('float32')  # попробуем предсказать на основе вектора запроса
predicted_class = classify(query_embedding, k=5)
print(predicted_class)

grade_improve


Проверим точность предсказания меток 

In [73]:
accuracy = sum(predicted == true for predicted, true in zip(predicted_labels, test_labels)) / len(test_labels)
print(f"Точность: {accuracy}")

Точность: 0.766704416761042
